In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import time

In [2]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument("--start-maximized")
options.add_argument("disable-infobars")
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\asus\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
url='https://www.metrocuadrado.com/'
driver.get(url)
url='https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-bellavista-3-habitaciones-4-banos-2-garajes/11079-M2838750'
driver.get(url)

In [79]:
def information_apt(url,information):
    driver.get(url)
    time.sleep(3) 
    apt=dict({'URL':url})
    for element in driver.find_elements_by_xpath('//div[@class="Col-sc-14ninbu-0 lfGZKA mb-3 pb-1 col-12 col-lg-3"]'):
        apt.update({element.text.split('\n')[0]:element.text.split('\n')[1]})
    print(apt)

    card_header=driver.find_elements_by_xpath('//div[@class="card-header"]')
    for card_index in range(len(card_header)-1) :
        time.sleep(2)  
        #List 
        l=list(set(map(lambda x: x.text,
                                   driver.find_elements_by_xpath('//div[@class="Col-sc-14ninbu-0 lfGZKA mb-3 col-12 col-md-3"]'))))
        #action
        action = ActionChains(driver)
        #Scroll
        driver.execute_script("arguments[0].scrollIntoView();", card_header[card_index+1])
        action.move_to_element(card_header[card_index+1]).click().perform()

        #List
        apt.update({card_header[card_index].text:l})
    l=list(set(map(lambda x: x.text,
                                   driver.find_elements_by_xpath('//div[@class="Col-sc-14ninbu-0 lfGZKA mb-3 col-12 col-md-3"]'))))
    apt.update({card_header[-1].text:l})

    temp=pd.DataFrame({a: str(b) for a, b in apt.items()},index=[0])

    information=information.append(temp)
    return information


In [80]:
information=pd.DataFrame()
information=information_apt('https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-bellavista-3-habitaciones-4-banos-2-garajes/11079-M2838750',information)
information=information_apt('https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-club-los-lagartos-3-habitaciones-2-banos-1-garajes/12503-3705639',information)


{'URL': 'https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-bellavista-3-habitaciones-4-banos-2-garajes/11079-M2838750', 'Código inmueble': '11079-M2838750', 'Barrio común': 'LOS ROSALES', 'Precio': '$1.480.000.000', 'Antigüedad': 'Entre 0 y 5 años', 'Área construida': '160 m²', 'Área privada': '160 m²', 'Valor administración': '$1.100.000', 'Parqueaderos': '2'}
{'URL': 'https://www.metrocuadrado.com/inmueble/venta-apartamento-bogota-club-los-lagartos-3-habitaciones-2-banos-1-garajes/12503-3705639', 'Código inmueble': '12503-3705639', 'Barrio común': 'El Rincón', 'Precio': '$202.000.000', 'Antigüedad': 'Entre 10 y 20 años', 'Área construida': '65 m²', 'Área privada': '65 m²', 'Valor administración': '$139.500', 'Parqueaderos': '1'}


In [81]:
information

,URL,Código inmueble,Barrio común,Precio,Antigüedad,Área construida,Área privada,Valor administración,Parqueaderos,Interiores,Exteriores,Del sector,Zonas comunes
0,https://www.metrocuadrado.com/inmueble/venta-a...,11079-M2838750,LOS ROSALES,$1.480.000.000,Entre 0 y 5 años,160 m²,160 m²,$1.100.000,2,"['', 'Zona de lavandería', 'Tipo de cortinas p...","['', 'Circuito cerrado de TV', 'Vigilancia 24h...","['', 'Cerca parques', 'Cerca centros comercial...",NaN
0,https://www.metrocuadrado.com/inmueble/venta-a...,12503-3705639,El Rincón,$202.000.000,Entre 10 y 20 años,65 m²,65 m²,$139.500,1,"['Tipo acabado de piso otro', '', 'Tipo de com...",NaN,NaN,"['', 'Salón comunal', 'Zonas verdes', 'Zona pa..."
